# Computation Theory

In [ ]:
#imports

import numpy as np

### Problem 1: Binary Words and Operations

In [ ]:
def Parity(x,y,z):
    x = np.int32(x)
    x = np.int32(y)
    x = np.int32(z)

    return np.uint32(x ^ y ^ z)

### Problem 2: Fractional Parts of Cube Roots

### Problem 3: Padding

### Problem 4: Hashes

### Problem 5: Passwords